In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

In [4]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [5]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [6]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [7]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [8]:
documents

[(['Hi', 'there'], 'greeting'),
 (['How', 'are', 'you'], 'greeting'),
 (['Is', 'anyone', 'there', '?'], 'greeting'),
 (['Hey'], 'greeting'),
 (['Hola'], 'greeting'),
 (['Hello'], 'greeting'),
 (['Good', 'day'], 'greeting'),
 (['Bye'], 'goodbye'),
 (['See', 'you', 'later'], 'goodbye'),
 (['Goodbye'], 'goodbye'),
 (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'),
 (['Till', 'next', 'time'], 'goodbye'),
 (['Thanks'], 'thanks'),
 (['Thank', 'you'], 'thanks'),
 (['That', "'s", 'helpful'], 'thanks'),
 (['Awesome', ',', 'thanks'], 'thanks'),
 (['Thanks', 'for', 'helping', 'me'], 'thanks'),
 (['How', 'you', 'could', 'help', 'me', '?'], 'options'),
 (['What', 'you', 'can', 'do', '?'], 'options'),
 (['What', 'help', 'you', 'provide', '?'], 'options'),
 (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'),
 (['What', 'support', 'is', 'offered'], 'options'),
 (['How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?'], 'adverse_drug'),
 (['Open', 'adverse', 'drugs', 'module'], 'adver

In [9]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [10]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])


In [11]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


Training data created


<ipython-input-11-9ffac2c3f218>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [14]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


In [15]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])


In [16]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


Epoch 1/200
10/10 [==============================] - 17s 4ms/step - loss: 2.2730 - accuracy: 0.0986
Epoch 2/200
10/10 [==============================] - 0s 3ms/step - loss: 2.2536 - accuracy: 0.2231
Epoch 3/200
10/10 [==============================] - 0s 3ms/step - loss: 2.2228 - accuracy: 0.1962
Epoch 4/200
10/10 [==============================] - 0s 2ms/step - loss: 2.2406 - accuracy: 0.1521
Epoch 5/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1535 - accuracy: 0.1761
Epoch 6/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1496 - accuracy: 0.1387
Epoch 7/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0801 - accuracy: 0.3003
Epoch 8/200
10/10 [==============================] - 0s 3ms/step - loss: 2.1454 - accuracy: 0.2130
Epoch 9/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1124 - accuracy: 0.2554
Epoch 10/200
10/10 [==============================] - 0s 3ms/step - loss: 2.1057 - accuracy: 0.1034
Epoch 11

10/10 [==============================] - 0s 2ms/step - loss: 1.1778 - accuracy: 0.7478
Epoch 84/200
10/10 [==============================] - 0s 3ms/step - loss: 1.3383 - accuracy: 0.7078
Epoch 85/200
10/10 [==============================] - 0s 2ms/step - loss: 1.2870 - accuracy: 0.7406
Epoch 86/200
10/10 [==============================] - 0s 2ms/step - loss: 1.0434 - accuracy: 0.8380
Epoch 87/200
10/10 [==============================] - 0s 2ms/step - loss: 1.0290 - accuracy: 0.7862
Epoch 88/200
10/10 [==============================] - 0s 2ms/step - loss: 1.0302 - accuracy: 0.8273
Epoch 89/200
10/10 [==============================] - 0s 2ms/step - loss: 1.2035 - accuracy: 0.6927
Epoch 90/200
10/10 [==============================] - 0s 2ms/step - loss: 1.0223 - accuracy: 0.7539
Epoch 91/200
10/10 [==============================] - 0s 2ms/step - loss: 1.0654 - accuracy: 0.8663
Epoch 92/200
10/10 [==============================] - 0s 2ms/step - loss: 1.0865 - accuracy: 0.7220
Epoch 93/200


10/10 [==============================] - 0s 2ms/step - loss: 0.4641 - accuracy: 0.8738
Epoch 165/200
10/10 [==============================] - 0s 2ms/step - loss: 0.4613 - accuracy: 0.9053
Epoch 166/200
10/10 [==============================] - 0s 2ms/step - loss: 0.3744 - accuracy: 0.9361
Epoch 167/200
10/10 [==============================] - 0s 2ms/step - loss: 0.4340 - accuracy: 0.9553
Epoch 168/200
10/10 [==============================] - 0s 2ms/step - loss: 0.5417 - accuracy: 0.8674
Epoch 169/200
10/10 [==============================] - 0s 2ms/step - loss: 0.6004 - accuracy: 0.9008
Epoch 170/200
10/10 [==============================] - 0s 2ms/step - loss: 0.4342 - accuracy: 0.9553
Epoch 171/200
10/10 [==============================] - 0s 2ms/step - loss: 0.3713 - accuracy: 0.9555
Epoch 172/200
10/10 [==============================] - 0s 2ms/step - loss: 0.3898 - accuracy: 0.8896
Epoch 173/200
10/10 [==============================] - 0s 2ms/step - loss: 0.4560 - accuracy: 0.9159
Epoc